In [37]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

Рассмотрим непараметрический метод анализа данных - с помощью логистической регрессии

Возмем для примера - всем известный и вдоль и поперек изученный датасет

In [2]:
df = pd.read_csv('titanic.csv')

In [4]:
df = df[['Survived','Pclass','Sex','Age']]

Модель без предиктора (Intercept only model)

In [29]:
model = sm.formula.glm("Survived ~ 1",
                       family=sm.families.Binomial(), data=df).fit()
print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                  156
Model:                            GLM   Df Residuals:                      155
Model Family:                Binomial   Df Model:                            0
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -100.63
Date:                Sun, 26 Apr 2020   Deviance:                       201.25
Time:                        16:53:07   Pearson chi2:                     156.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.6360      0.168     -3.779      0.0

Получаем один коэфициент. Но, что показывает данное значение?

In [45]:
model.params.round(3)

Intercept   -0.636
dtype: float64

Данный коэфициент показывает распределение переменной Survived. А точнее логарифм шансов. В этом легко убедиться.<br>
Для начала взглянем на распределение переменной Survived

In [36]:
pd.DataFrame(df.Survived.value_counts())

,Survived
0,102
1,54


Итак погибло 102 пассажира, выжило 54.<br>
Расчитаем шансы на выживаемость(отношение вероятности положительного исхода к вероятности отрицательного исхода)<br>
odds = p/1-p.<br>
Вероятность положительного исхода 54/156 <br>
Вероятность отрицательного исхода 102/156 <br>

In [46]:
#odds = p/1-p
odds = (54/156)/(102/156)
np.log(odds).round(3)

-0.636

Теперь обратим внимание, что данный показатель число отрицательное и статистически значимо отличается от нуля P>|z|. Это позволяет нам сейчас сделать кое-какой вывод- логарифм odds значимо меньше нуля-следовательно шансы значимо меньше единицы и отсюда следует, что вероятность положительного исхода, меньше чем отрицательного.

Добавим один предиктор - 'Sex'

In [50]:
model1 = sm.formula.glm("Survived ~ Sex",
                       family=sm.families.Binomial(), data=df).fit()
print(model1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Survived   No. Observations:                  156
Model:                            GLM   Df Residuals:                      154
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -73.999
Date:                Sun, 26 Apr 2020   Deviance:                       148.00
Time:                        17:24:12   Pearson chi2:                     156.
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.9163      0.296      3.098      

Получаем  значения, но как их интерпретировать?
Для того, чтобы разобраться построим распределение переменной Survived относительно пола пассажира

In [70]:
pd.crosstab(df['Survived'], df['Sex'])

Sex,female,male
Survived,,
0,16,86
1,40,14


Расчитаем отдельно шансы выживания для мужчин и для женщин

In [76]:
odds_female = 40/16
odds_male = 14/86

Посчитаем логарифм шансов для женщин и для мужчин

In [77]:
ln_odds_male = np.log(odds_male)
ln_odds_female = np.log(odds_female)
print('ln_odds_male:',ln_odds_male)
print('ln_odds_female:',ln_odds_female)

ln_odds_male: -1.8152899666382492
ln_odds_female: 0.9162907318741551


Значение ln_odds_female это значение Intercept модели. <br>
Значение второго коэфициента - это значение логарифма отношения шансов для мужчин и женщин.

In [82]:
ratio_odds = odds_male/odds_female
np.log(ratio_odds)

-2.731580698512404

В общем виде равнение примет вид:<br>
    log(p/1-p) = 0.916 - 2.73*sex-Male* <br>
Допустим, что пассажир женщина тогда логарифм отношения шансов равен:<br>
    log(p/1-p) = 0.916 - 2.73 * 0 = 0.916<br>
А если мужчина, то шансы снижаются:<br>
    log(p/1-p) = 0.916 - 2.73 * 1 = -1.814<br>